# 优化卷积核技术
在实际的卷积训练中，为了加快速度，尝尝把卷积核裁开。比如一个3x3的过滤网，可以裁切成3x1和1x3两个过滤器，分别对原有输入做卷积操作，这样大大提升运算的速度。

> 原理：在浮点运算中乘法消耗资源比较多，我们目的就是尽量减小乘法运算。

- 比如对一个5x2的原始图片进行一次3x3的同卷积，相当于生成的5x2像素中每一次都要经历3x3次乘法，那么一共是90次。
- 同样是这个图片，如果先进行一次3x1的同卷积需要30次运算，再进行一次1x3的同卷积还是30次，一共才60次。

随着张量维度的增大，层数的增多，减少的运算会更多。并且有公式来保证3x1的矩阵乘上1x3的矩阵会正好生成3x3的矩阵。

# 多通道卷积技术演示
多通道卷积，可以理解为一种新型的CNN网络模型，在原有卷积模型基础上的扩展。
- 原有的卷积层中是使用单个尺寸的卷积核对输入数据卷积操作，生成若干个feature map。
- 而多通道卷积的变化就是，在单个卷积层中加入若干个不同尺寸的过滤器，这样会使生成的feature map特征更加多样性。
![](imgs/23_multichannel convolution.png)

为网络的卷积层增加不同尺寸的卷积核。这里将原有的5x5卷积，扩展到7x7卷积、1x1卷积、3x3卷积，并通过concat函数并在一起。

In [ ]:
x_image = tf.reshape(x, [-1,24,24,3])
 
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
#######################################################多卷积核
W_conv2_5x5 = weight_variable([5, 5, 64, 64]) 
b_conv2_5x5 = bias_variable([64]) 
W_conv2_7x7 = weight_variable([7, 7, 64, 64]) 
b_conv2_7x7 = bias_variable([64]) 
 
W_conv2_3x3 = weight_variable([3, 3, 64, 64]) 
b_conv2_3x3 = bias_variable([64]) 
 
W_conv2_1x1 = weight_variable([3, 3, 64, 64]) 
b_conv2_1x1 = bias_variable([64]) 
 
h_conv2_1x1 = tf.nn.relu(conv2d(h_pool1, W_conv2_1x1) + b_conv2_1x1)
h_conv2_3x3 = tf.nn.relu(conv2d(h_pool1, W_conv2_3x3) + b_conv2_3x3)
h_conv2_5x5 = tf.nn.relu(conv2d(h_pool1, W_conv2_5x5) + b_conv2_5x5)
h_conv2_7x7 = tf.nn.relu(conv2d(h_pool1, W_conv2_7x7) + b_conv2_7x7)
h_conv2 = tf.concat([h_conv2_5x5,h_conv2_7x7,h_conv2_3x3,h_conv2_1x1],3)
 
h_pool2 = max_pool_2x2(h_conv2)
#######################################################
W_conv3 = weight_variable([5, 5, 256, 10])
b_conv3 = bias_variable([10])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
 
nt_hpool3=avg_pool_6x6(h_conv3)#10
nt_hpool3_flat = tf.reshape(nt_hpool3, [-1, 10])
y_conv=tf.nn.softmax(nt_hpool3_flat)

# 批量归一化
还有一种十分广泛的优化方法--**批量归一化（简称BN算法）**。一般用在**全连接**或**卷积神经网络**。这个方法使神经网络的识别准确度上升了一个台阶。
## 1.批量归一化介绍
假如有一个极简的网络模型，每一层只有一个节点，没有偏置。那么如果这个网络有三层的话，可以用如下式子表示其输出值：

假设有两个神经网络，学习出了两套权重（w1：1，w2：1，w3：1）和（w1：0.01，w2：10000，w3：0.01），它们对应的输出z都是相同的。现在让它们训练一次，看看会发生什么。

（1）反向传播：假设反向传播时计算出的损失值Δy为1，那么对于这两套权重的修正值将变为（Δw1：1，Δw2：1，Δw3：1）和（Δw1：100，Δw2：0.0001，Δw3：100）。

（2）更新权重：这时更新过后的两套权重就变成了（w1：2，w2：2，w3：2）和（w1：100.01，w2：10000.0001，w3：100.01）。
（3）第二次正向传播：假设输入样本是1，第一个神经网络的值为：

Z=1×2×2×2 = 8

第二个神经网络的值为：

Z=1×100.01×10000.0001×100.01=100000000

可以看到两个网络的输出差值差别巨大。如果再往下进行，这时计算出的loss值会变得更大，使得网络无法计算，这种现象也叫做**梯度爆炸**。产生梯度爆炸的原因就是**因为网络的内部协变量转移(Internal Covariate Shift)，即正向传播时的不同层的参数会将反向训练计算时所参照的数据样本分布改变。**

这就是引入批量正则化的目的。它的作用是要最大限度地保证每次的正向传播输出在同一分布上，这样反向计算时参照的数据分布就会与正向计算时的数据分布一样了。保证了分布统一，对权重的调整才会更有意义。

**批量正则化的做法就是将每一层运算出来的数据都归一化成均值为0方差为1的标准高斯分布。这样就会在保留样本分布特征的同时，又消除了层与层间的分布差异。**

> 在实际应用中，批量归一化的收敛非常快，并且具有很强的泛化能力，**某种情况下可以完全替代前面讲过的正则化、Dropout**。

## 2.批量归一化定义
TensorFlow中自带BN函数定义：
___
```python
tf.nn.batch_normalization(x,
                         mean,
                         variance,
                         offset,
                         scale,
                         variance_epsilon,
                         name=None)
```
___

- x:代表输入。
- mean:代表样本的均值。
- variance:代表方差。
- offset:代表偏移，即相加一个转化值，后面我们会用激活函数来转换，所以这里不需要再转化，直接使用0。
- scale:代表缩放，即乘以一个转化值，同理，一般都用1。
- variance_epsilon:是为了避免分母为0的情况，给分母加一个极小值。默认即可。

想要使用这个函数，必须由另一个函数配合--tf.nn.moments，由它来计算均值和方差，然后就可以使用BN了。tf.nn.moments定义如下:
___
```python
tf.nn.moments(x,axes,name=None,keep_dims=False)
```
___
axes主要是指定哪个轴来求均值和方差。

> axes在使用过程中经常容易犯错。为了求样本的均值和方差，一般都会设为保留最后一个维度，对于x来讲可以直接使用公式axis = list(range(len(x.get_shape())-1))即可。例如，[128,3,3,12]axes就为[0,1,2]，输出的均值方差维度为[12]。

为了更好的效果，我们使用平滑指数衰减的方法来优化每次的均值与方差，于是就用到了tf.train.ExponentialMovingAverage函数。它的作用是让上一次的值对本次的值有个衰减后的影响，从而使每次的值连起来后会相对平滑一些。
___
```python
shadow_variable = decay * shadow_variable + (1-decay) * variable
```
___

- decay:代表衰减指数，是在ExponentialMovingAverage中指定的，比如0.9。
- variable:代表本批次样本中的值。
- 等式右边的shadow_variable:代表上次总样本的值。
- 等式左边shadow_variable:代表计算出来的本次总样本的值。

## 3.批量归一化的简单用法
TensorFlow中layers模块里面又实现了一次BN函数，相当于把几个函数合并到了一起，使用起来更方便。

In [1]:
from tensorflow.contrib.layers.python.layers import batch_norm

定义如下：
___
```python
def batch_norm(inputs,
               decay=0.999,
               center=True,
               scale=False,
               epsilon=0.001,
               activation_fn=None,
               param_initializers=None,
               param_regularizers=None,
               updates_collections=ops.GraphKeys.UPDATE_OPS,
               is_training=True,
               reuse=None,
               variables_collections=None,
               outputs_collections=None,
               trainable=True,
               batch_weights=None,
               fused=False,
               data_format=DATA_FORMAT_NHWC,
               zero_debias_moving_mean=False,
               scope=None,
               renorm=False,
               renorm_clipping=None,
               renorm_decay=0.99):
```
___

- inputs:输入。
- 代表移动平均值的衰败速度，是使用了一种叫做平滑指数衰减的方法更新均值方差，一般会设为0.9；值太小会导致均值和方差更新太快，而值太大又会导致几乎没有衰减，容易出现过拟合，这种情况一般需要把值调小点。
- scale:是否进行变化（通过乘一个gamma值进行缩放），我们常习惯在BN后面接着一个线性的变化，如Relu。所以scale一般都会设为False。因为后面有对数据的转化处理，因此这里就不用再处理了。
- epsilon:是为了避免分母为0的情况，给分母加一个极小值。一般默认即可。
- is_training：当它为True时，代表是训练过程，这时会不断更新样本集的均值与方差。当测试时，要设成False，这样就会使用训练样本集的均值与方差。
- updates_collections：其变量默认是tf.GraphKeys.UPDATE_OPS，在训练时提供了一种内置的均值方差更新机制，即通过图（一个计算任务）中的tf.GraphKeys. UPDATE_OPS变量来更新。但它是在每次当前批次训练完成后才更新均值和方差，这样导致当前数据总是使用前一次的均值和方差，没有得到最新的更新。所以一般都会将其设成None，让均值和方差即时更新。这样做虽然相比默认值在性能上稍慢点，但是对模型的训练还是有很大帮助的。
- reuse：支持共享变量，与下面的scope参数联合使用。
- scope：指定变量的作用域variable_scope。

# 为CIFAR图片分类模型添加BN